In [3]:
import os
import h5py
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np
import fixmat as ft

In [9]:
#filename = '/Users/Lauren/cahv/data/etdb.hdf5'
#f = h5py.File(filename, 'r')
#baseline,meta = ft.load('/Users/Lauren/cahv/data/etdb.hdf5', 'Baseline')
#appc,meta_appc = ft.load('/Users/Lauren/cahv/data/APP_raw.hdf', "APP")
eeg,meta_eeg = ft.load('/Users/Lauren/eye-movements/data/hdf/EEG_raw.hdf', "EEG")

In [17]:
img_idx = range(1, len(os.listdir("/Users/Lauren/eye-movements/data/14/"))+1)

for i in img_idx:
    test_img_data = eeg.groupby("filenumber").get_group(i)
    img = cv2.imread("/Users/Lauren/eye-movements/data/14/" + str(i) + ".jpg")
    for idx,row in test_img_data.iterrows():
        cv2.circle(img, (int(row["x"]),int(row["y"])),7,color=(0,255,0))
        
    cv2.imwrite("/Users/Lauren/eye-movements/data/viz/14/" + str(i) + ".jpg", img)

KeyboardInterrupt: 